In [2]:
import sys
import os
import functools
import scipy
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pylab as pylab
from matplotlib.colors import LogNorm
import pickle
import math
import subprocess
import warnings

sys.path.insert(2, "/global/projecta/projectdirs/lux/data")
sys.path.insert(0,'/project/projectdirs/lux/Tools/LUXcode/Trunk/DataAnalysis/Python/' )
sys.path.insert(0,'/project/projectdirs/lux/Tools/LUXcode/Trunk/DataAnalysis/Python/aLib' )
import __init__ as S
import dp as dp
from scipy.optimize import OptimizeWarning

warnings.simplefilter("error", OptimizeWarning)
pylab.rcParams["figure.figsize"]=15,8
np.seterr(invalid='ignore')


{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [3]:
# Tritium September 2015
#ts2 = np.load("/global/homes/w/weiji/ShaunDataSets/ts5_2.pickle")
#ts3 = np.load("/global/homes/w/weiji/ShaunDataSets/ts5_3.pickle")
#ts4 = np.load("/global/homes/w/weiji/ShaunDataSets/ts5_4.pickle")
#ts5 = np.load("/global/homes/w/weiji/ShaunDataSets/ts5_5.pickle")
#ts6 = np.load("/global/homes/w/weiji/ShaunDataSets/ts5_6.pickle")

In [9]:
ts4= np.load("/project/projectdirs/lux/users/weiji/data/201509tritiumReal.npy")[0]

In [10]:
ts4


Dictionary contents:
                             FIELDS: TYPE                             CONTENTS
                             ------  ----                             --------
                           RQ_paths: [1 list],                         
                            fileNum: [(12849,) uint16array],          [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1...
                        file_number: [(268108,) uint32array],         [6081 6081 6081 6081 6081 6081...
                        numCutsPass: [1 list],                         
                     numEventsTotal: [1 int],                         222381
                            pathNum: [(12849,) uint8array],           [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
                     pulse_area_phe: [(10, 268108) float32array],     [  3.39977422e+04   3.26255035...
               pulse_classification: [(10, 268108) uint32array],      [1 1 1 1 1 1 1 1 1 1 3 3 1 1 4...
                       s1s2_pairing: [(10, 268108) uint32array],      [1 1 1 1 1

In [11]:
ss4 = np.load("/project/projectdirs/lux/users/weiji/data/201509tritium.npy")[0]

In [12]:
ss4

Dictionary contents:
                             FIELDS: TYPE                             CONTENTS
                             ------  ----                             --------
                           RQ_paths: [1 list],                         
                            fileNum: [(260095,) uint16array],         [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
                        file_number: [(260095,) uint32array],         [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2...
                        numCutsPass: [1 list],                         
                     numEventsTotal: [1 int],                         353364
                            pathNum: [(260095,) uint8array],          [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
                     pulse_area_phe: [(10, 260095) float32array],     [  2.29091525  25.86428452  29...
               pulse_classification: [(10, 260095) uint32array],      [1 1 1 1 1 3 1 1 1 1 1 1 1 1 1...
                       s1s2_pairing: [(10, 260095) uint32array],      [1 1 1 1 1

In [13]:
Efieldmap = np.loadtxt("/project/projectdirs/lux/users/weiji/Run4FieldMaps/v10_fields_20150928T2324_cp17544.txt")
Efieldmap

array([[-24. , -24. ,   5.8, ...,   nan,   nan,   0. ],
       [-23. , -24. ,   5.8, ...,   nan,   nan,   0. ],
       [-22. , -24. ,   5.8, ...,   nan,   nan,   0. ],
       ..., 
       [ 22. ,  24. ,  53.8, ...,   nan,   nan,   0. ],
       [ 23. ,  24. ,  53.8, ...,   nan,   nan,   0. ],
       [ 24. ,  24. ,  53.8, ...,   nan,   nan,   0. ]])

In [14]:
Efieldmap.shape

(117649, 8)

In [15]:
#define a funtion do band's mean value and sigma calculation
def f(x, a, b, c):
    return a * np.exp(-(x - b)**2.0 / (2 * c**2))

def bandfit(x, y, bins=10, range=None, normed=False, weights=None):
    H, xedges,yedges = np.histogram2d(x, y, bins, range, normed, weights)
    log10xlist=[]
    mu=[]
    sigma=[]
    for ii in xrange(0,xedges.shape[0]-1):
        try:
            log10y = [0.5 * (yedges[i] + yedges[i+1]) for i in xrange(yedges.shape[0]-1)]
            Hx = H[ii,:]
            popt, pcov = sp.optimize.curve_fit(f, log10y, Hx)
            log10xlist.append(0.5 * (xedges[ii] + xedges[ii+1]))
            mu.append(popt[1])
            sigma.append(popt[2])
        except BaseException:
            print "Fitting data in bin ",ii,"failed."
    return(np.array(mu),np.abs(sigma),np.array(log10xlist))


In [16]:
def LoadXYZDependentEField(EFieldMap, NumxyzDependentEFieldVars = 8, MinLocationX = -24.0 ,NumPlanesX = 49, StepSizeX = 1. ,MinLocationY = -24.00 ,NumPlanesY = 49, StepSizeY = 1.,MinLocationZ = 5.8 ,NumPlanesZ = 49, StepSizeZ = 1.):
    EFieldMapMatrix = np.zeros((NumPlanesX,NumPlanesY,NumPlanesZ,NumxyzDependentEFieldVars))
    for ii in xrange(0,EFieldMap.shape[0]):
        xx = int(round((EFieldMap[ii,0]-MinLocationX)/StepSizeX))
        yy =int(round((EFieldMap[ii,1]-MinLocationY)/StepSizeY))
        zz =int(round((EFieldMap[ii,2]-MinLocationZ)/StepSizeZ))
        for vv in xrange(0,NumxyzDependentEFieldVars):
            if vv == 2:
                EFieldMapMatrix[xx,yy,zz,vv] = EFieldMap[ii,vv] 
            elif vv==6:
                EFieldMapMatrix[xx,yy,zz,vv] = EFieldMap[ii,vv]
            else:
                EFieldMapMatrix[xx,yy,zz,vv] = EFieldMap[ii,vv]
            
                
    return EFieldMapMatrix,NumxyzDependentEFieldVars,((MinLocationX,NumPlanesX,StepSizeX),(MinLocationY,NumPlanesY,StepSizeY),(MinLocationZ,NumPlanesZ,StepSizeZ))

def GetXYZDependentElectricField(x,LoadEFieldMap,ifprint=0):
    EFieldMapMatrix,NumxyzDependentEFieldVars,((MinLocationX,NumPlanesX,StepSizeX),(MinLocationY,NumPlanesY,StepSizeY),(MinLocationZ,NumPlanesZ,StepSizeZ))= LoadEFieldMap
    xx =int(round((x[0]-MinLocationX)/StepSizeX))
    yy =int(round((x[1]-MinLocationY)/StepSizeY))
    if (xx<0 or xx>NumPlanesX-2 or yy<0 or yy>NumPlanesY-2):
#   if 0:
        return -100000
    else:
        distance2 = (x[0]-EFieldMapMatrix[xx,yy,0,0])**2+(x[1]-EFieldMapMatrix[xx,yy,0,1])**2
        for xxx in xrange(xx,xx+1):
            for yyy in xrange(yy,yy+1):                
                    if distance2 > ((x[0]-EFieldMapMatrix[xxx,yyy,0,0])**2+(x[1]-EFieldMapMatrix[xxx,yyy,0,1])**2):                                               
                        xx = xxx
                        yy = yyy
        zz=0
#        print xx,yy
        dist = np.abs(x[2]-EFieldMapMatrix[xx,yy,0,6])
#        print dist
        for zzz in xrange(0,NumPlanesZ):
            if np.abs(x[2]-EFieldMapMatrix[xx,yy,zzz,6])<dist:
                dist = np.abs(x[2]-EFieldMapMatrix[xx,yy,zzz,6])
                zz=zzz
    if ifprint:
        print "index_x,y,z,E",xx,yy,zz,EFieldMapMatrix[xx,yy,zz,3]
        print "x,y,z,",EFieldMapMatrix[xx,yy,zz,0],EFieldMapMatrix[xx,yy,zz,1],EFieldMapMatrix[xx,yy,zz,2]
        print "S2x,S2y,t,",EFieldMapMatrix[xx,yy,zz,4],EFieldMapMatrix[xx,yy,zz,5],EFieldMapMatrix[xx,yy,zz,6]
    return EFieldMapMatrix[xx,yy,zz,3]

def GetXYZDependentElectricField_s2(x,LoadEFieldMap,ifprint=0):
    EFieldMapMatrix,NumxyzDependentEFieldVars,((MinLocationX,NumPlanesX,StepSizeX),(MinLocationY,NumPlanesY,StepSizeY),(MinLocationZ,NumPlanesZ,StepSizeZ))= LoadEFieldMap    
    try:
        xx=0
        yy=0
        zz=0
        distance2 = 10000
        for xxx in xrange(0,NumPlanesX):
            for yyy in xrange(0,NumPlanesY):  
                 for zzz in xrange(0,NumPlanesZ):
                    if distance2 > ((x[0]-EFieldMapMatrix[xxx,yyy,zzz,4])**2+(x[1]-EFieldMapMatrix[xxx,yyy,zzz,5])**2)+(x[2]-EFieldMapMatrix[xxx,yyy,zzz,6])**2/144.:                                               
                        xx = xxx
                        yy = yyy
                        zz = zzz
                        distance2 = (x[0]-EFieldMapMatrix[xxx,yyy,zzz,4])**2+(x[1]-EFieldMapMatrix[xxx,yyy,zzz,5])**2+(x[2]-EFieldMapMatrix[xxx,yyy,zzz,6])**2/144.
        if ifprint:
            print "index_x,y,z,E",xx,yy,zz,EFieldMapMatrix[xx,yy,zz,3]
            print "x,y,z,",EFieldMapMatrix[xx,yy,zz,0],EFieldMapMatrix[xx,yy,zz,1],EFieldMapMatrix[xx,yy,zz,2]
            print "S2x,S2y,t,",EFieldMapMatrix[xx,yy,zz,4],EFieldMapMatrix[xx,yy,zz,5],EFieldMapMatrix[xx,yy,zz,6]

            print distance2
        return EFieldMapMatrix[xx,yy,zz,:]
    except :
        return -100000

In [18]:
def errorf(x,*args):
    (S2x,S2y,T),(x0,y0,z0),(s2x0,s2y0,t0),(ds2xdx,ds2xdy,ds2xdz),(ds2ydx,ds2ydy,ds2ydz),(dtdx,dtdy,dtdz) = args
    return (S2x - (s2x0 + (x[0]-x0)*ds2xdx + (x[1]-y0)*ds2xdy + (x[2]-z0)*ds2xdz))**2 \
            +(S2y - (s2y0 + (x[0]-x0)*ds2ydx + (x[1]-y0)*ds2ydy + (x[2]-z0)*ds2ydz))**2 \
            +(T - (t0 + (x[0]-x0)*dtdx + (x[1]-y0)*dtdy + (x[2]-z0)*dtdz))**2 

def intepolation(x,x0,f0,dfdx):
    x=np.array(x)
    x0 = np.array(x0)
    dfdx = np.array(dfdx)
    return f0+np.sum(dfdx*(x-x0))

def GetXYZFromXYZS2TDependentElectricField((S2x,S2y,DriftTime),LoadEFieldMap,ifprint=0):
    EFieldMapMatrix,NumxyzDependentEFieldVars,((MinLocationX,NumPlanesX,StepSizeX),(MinLocationY,NumPlanesY,StepSizeY),(MinLocationZ,NumPlanesZ,StepSizeZ))= LoadEFieldMap    
    """
        This function is getting real x y z E from known S2x,S2y,DriftTime,fieldmap
        S2x, S2y in cm
        DriftTime in us
    """
    warnings.filterwarnings('error')
    try:
        xx=25
        yy=25
        zz=47
        attempt=200
        notsuccess = 1
        while((attempt > 0) and notsuccess):
            attempt-=1
            notsuccess = 0
            
            if S2x< EFieldMapMatrix[xx,yy,zz,4]:
                notsuccess = 1
                xx -= 1
                if ifprint:
                    print "xx--"
            elif S2x > EFieldMapMatrix[xx+1,yy,zz,4]:
                notsuccess = 1
                xx += 1
                if ifprint:
                    print "xx++"
            elif S2y< EFieldMapMatrix[xx,yy,zz,5]:
                notsuccess = 1
                yy -= 1
                if ifprint:
                    print "yy--"
            elif S2y > EFieldMapMatrix[xx,yy+1,zz,5]:
                notsuccess = 1
                yy += 1
                if ifprint:
                    print "yy++"
            elif DriftTime < EFieldMapMatrix[xx,yy,zz+1,6]:
                notsuccess = 1
                zz += 1
                if ifprint:
                    print "zz--"
            elif DriftTime > EFieldMapMatrix[xx,yy,zz,6]:
                notsuccess = 1
                zz -= 1
                if ifprint:
                    print "zz++"
            if xx<0 or xx >NumPlanesX-2 or yy<0 or yy >NumPlanesY-2 or zz<0 or zz >NumPlanesZ-2:
                if ifprint:
                    print "(S2x =",S2x ,"cm, S2y =",S2y ,"cm, DriftTime =",DriftTime,"ns ) is OUT of range in the field map."
                return (np.nan,np.nan,np.nan,np.nan,S2x,S2y,DriftTime,np.nan)
            if ifprint:
                print "attempt",attempt
                print "xx,yy,zz",xx,yy,zz
                print "Not success",notsuccess
        if ifprint:
            print "attempt finished"
        if attempt ==0 and ifprint:
            print "Did not find a point in the map matches with (S2x =",S2x ,"cm, S2y =",S2y ,"cm, DriftTime =",DriftTime,"ns )"
            print "One of the closest point is (S2x =",EFieldMapMatrix[xx,yy,zz,4],"cm, S2y =",EFieldMapMatrix[xx,yy,zz,5],"cm, DriftTime =",EFieldMapMatrix[xx,yy,zz,6],"ns )"
            print "Check whether you are using the right field map and right S2x, S2y,S2z",attempt
        if xx<0 or xx >NumPlanesX-2 or yy<0 or yy >NumPlanesY-2 or zz<0 or zz >NumPlanesZ-2 or\
            np.isnan(EFieldMapMatrix[xx,yy,zz,3])            :
            if ifprint:
                print "(S2x =",S2x ,"cm, S2y =",S2y ,"cm, DriftTime =",DriftTime,"ns ) is OUT of range in the field map."
            return (np.nan,np.nan,np.nan,np.nan,S2x,S2y,DriftTime,np.nan)
        def dfdx(xx,yy,zz,num):
            dfdx = 1./4./StepSizeX*(-EFieldMapMatrix[xx,yy,zz,num]-EFieldMapMatrix[xx,yy,zz+1,num]\
                     -EFieldMapMatrix[xx,yy+1,zz,num]-EFieldMapMatrix[xx,yy+1,zz+1,num]\
                     +EFieldMapMatrix[xx+1,yy,zz,num]+EFieldMapMatrix[xx+1,yy,zz+1,num]\
                     +EFieldMapMatrix[xx+1,yy+1,zz,num]+EFieldMapMatrix[xx+1,yy+1,zz+1,num])
            dfdy = 1./4./StepSizeY*(-EFieldMapMatrix[xx,yy,zz,num]-EFieldMapMatrix[xx,yy,zz+1,num]\
                     +EFieldMapMatrix[xx,yy+1,zz,num]+EFieldMapMatrix[xx,yy+1,zz+1,num]\
                     -EFieldMapMatrix[xx+1,yy,zz,num]-EFieldMapMatrix[xx+1,yy,zz+1,num]\
                     +EFieldMapMatrix[xx+1,yy+1,zz,num]+EFieldMapMatrix[xx+1,yy+1,zz+1,num])
            dfdz = 1./4./StepSizeZ*(-EFieldMapMatrix[xx,yy,zz,num]+EFieldMapMatrix[xx,yy,zz+1,num]\
                     -EFieldMapMatrix[xx,yy+1,zz,num]+EFieldMapMatrix[xx,yy+1,zz+1,num]\
                     -EFieldMapMatrix[xx+1,yy,zz,num]+EFieldMapMatrix[xx+1,yy,zz+1,num]\
                     -EFieldMapMatrix[xx+1,yy+1,zz,num]+EFieldMapMatrix[xx+1,yy+1,zz+1,num])
            return (dfdx,dfdy,dfdz)
        (ds2xdx,ds2xdy,ds2xdz) = dfdx(xx,yy,zz,4)
        (ds2ydx,ds2ydy,ds2ydz) = dfdx(xx,yy,zz,5)
        (dtdx,dtdy,dtdz)   = dfdx(xx,yy,zz,6)
        (xreal0,yreal0,zreal0) = (EFieldMapMatrix[xx,yy,zz,0],EFieldMapMatrix[xx,yy,zz,1],EFieldMapMatrix[xx,yy,zz,2])
        (s2x0,s2y0,t0) = (EFieldMapMatrix[xx,yy,zz,4],EFieldMapMatrix[xx,yy,zz,5],EFieldMapMatrix[xx,yy,zz,6])
           
        result= sp.optimize.fmin(errorf,x0=(xreal0,yreal0,zreal0),args = ((S2x,S2y,DriftTime),(xreal0,yreal0,zreal0),(s2x0,s2y0,t0),(ds2xdx,ds2xdy,ds2xdz),(ds2ydx,ds2ydy,ds2ydz),(dtdx,dtdy,dtdz)),disp=0)
        if ifprint:
            print result
        (xreal,yreal,zreal) = result
        E = intepolation((xreal,yreal,zreal),(xreal0,yreal0,zreal0),EFieldMapMatrix[xx,yy,zz,3],dfdx(xx,yy,zz,3))
        
        if ifprint:
            print "index_x,y,z,E",xx,yy,zz,EFieldMapMatrix[xx,yy,zz,3]
            print xreal,yreal,zreal,E,S2x,S2y,DriftTime,EFieldMapMatrix[xx,yy,zz,7]

        
    except BaseException:
        return (np.nan,np.nan,np.nan,np.nan,S2x,S2y,DriftTime,np.nan)
    #return (np.nan,np.nan,np.nan,np.nan,S2x,S2y,DriftTime,np.nan)
    return (xreal,yreal,zreal,E,S2x,S2y,DriftTime,EFieldMapMatrix[xx,yy,zz,7])

In [19]:
LoadEFieldMap = LoadXYZDependentEField(Efieldmap)

In [20]:
EFieldMapMatrix,NumxyzDependentEFieldVars,((MinLocationX,NumPlanesX,StepSizeX),(MinLocationY,NumPlanesY,StepSizeY),(MinLocationZ,NumPlanesZ,StepSizeZ))= LoadEFieldMap

In [21]:
GetXYZFromXYZS2TDependentElectricField((-2,-3,255),LoadEFieldMap,ifprint=1)


xx--
attempt 199
xx,yy,zz 24 25 47
Not success 1
xx--
attempt 198
xx,yy,zz 23 25 47
Not success 1
xx--
attempt 197
xx,yy,zz 22 25 47
Not success 1
yy--
attempt 196
xx,yy,zz 22 24 47
Not success 1
yy--
attempt 195
xx,yy,zz 22 23 47
Not success 1
yy--
attempt 194
xx,yy,zz 22 22 47
Not success 1
xx--
attempt 193
xx,yy,zz 21 22 47
Not success 1
yy--
attempt 192
xx,yy,zz 21 21 47
Not success 1
yy--
attempt 191
xx,yy,zz 21 20 47
Not success 1
xx++
attempt 190
xx,yy,zz 22 20 47
Not success 1
zz++
attempt 189
xx,yy,zz 22 20 46
Not success 1
yy++
attempt 188
xx,yy,zz 22 21 46
Not success 1
xx--
attempt 187
xx,yy,zz 21 21 46
Not success 1
yy--
attempt 186
xx,yy,zz 21 20 46
Not success 1
xx++
attempt 185
xx,yy,zz 22 20 46
Not success 1
yy++
attempt 184
xx,yy,zz 22 21 46
Not success 1
xx--
attempt 183
xx,yy,zz 21 21 46
Not success 1
yy--
attempt 182
xx,yy,zz 21 20 46
Not success 1
xx++
attempt 181
xx,yy,zz 22 20 46
Not success 1
yy++
attempt 180
xx,yy,zz 22 21 46
Not success 1
xx--
attempt 179
xx,

(-2.2531920413873125,
 -2.5207311092611118,
 2.7999969659998634,
 -3354591.4410441644,
 -2,
 -3,
 255,
 1.0)

In [22]:
EFieldMapMatrix[22][20][46]

array([ -2.00000000e+00,  -4.00000000e+00,   5.18000000e+01,
         4.15144840e+04,  -2.00100000e+00,  -3.98400000e+00,
         1.00000000e+01,   1.00000000e+00])

In [23]:
EFieldMapMatrix[22][21][46]

array([ -2.00000000e+00,  -3.00000000e+00,   5.18000000e+01,
         4.11980970e+04,  -1.99000000e+00,  -3.00800000e+00,
         1.00000000e+01,   1.00000000e+00])

In [24]:
EFieldMapMatrix[21][21][46]

array([ -3.00000000e+00,  -3.00000000e+00,   5.18000000e+01,
         4.11980970e+04,  -2.99100000e+00,  -2.99600000e+00,
         1.00000000e+01,   1.00000000e+00])

In [25]:
EFieldMapMatrix[21][20][46]

array([ -3.00000000e+00,  -4.00000000e+00,   5.18000000e+01,
         4.15510890e+04,  -2.99500000e+00,  -3.99100000e+00,
         1.00000000e+01,   1.00000000e+00])

In [26]:
#GetXYZDependentElectricField((raw_x_ts4[2],raw_x_ts4[2],drift_time_ts4[2]),LoadEFieldMap,ifprint=1)
#print raw_x_ts4[2],raw_x_ts4[2],drift_time_ts4[2]

In [27]:
mask_S1_ts4 = (ts4.pulse_classification==1)&(ts4.s1s2_pairing==1)
mask_S2_ts4 = (ts4.pulse_classification==2)&(ts4.s1s2_pairing==1)

In [28]:
S1_ts4 = (ts4.xyz_corrected_pulse_area_all_phe*mask_S1_ts4).sum(0)
S2_ts4 = (ts4.xyz_corrected_pulse_area_all_phe*mask_S2_ts4).sum(0)

In [29]:
raw_x_ts4 = (ts4.x_cm*mask_S2_ts4).sum(0) #this is ok since i have verified there is only
raw_y_ts4 = (ts4.y_cm*mask_S2_ts4).sum(0) #one S2 in each event that has made it through cuts
raw_x_del_ts4 = (ts4.x_cm_del*mask_S2_ts4).sum(0) #this is ok since i have verified there is only
raw_y_del_ts4 = (ts4.y_cm_del*mask_S2_ts4).sum(0) #one S2 in each event that has made it through cuts
drift_time_ts4 = (ts4.z_drift_samples*mask_S2_ts4).sum(0)/100.
negative_drift_time_ts4 = [0-a for a in drift_time_ts4]
raw_r_ts4 = (raw_x_ts4**2 + raw_y_ts4**2)**(1./2)# mask out only tritium_energies this is a mask on an entire event not a pulse
# possibly add a radial cut
# possibly add a drift time cut
mask_tritium_energy_ts4 = (np.log10(S1_ts4)<2.5)&(np.log10(S2_ts4)<5.0)&(3.1<np.log10(S2_ts4))&(6.1>np.log10(S2_ts4)+np.log10(S1_ts4))
tS1_ts4 = []
tS2_ts4 = []
traw_x_ts4 = []
traw_y_ts4 = []
traw_r_ts4 = []
tdrift_time_ts4 = []
x_del_ts4=[]
y_del_ts4=[]
for aa in range(len(mask_tritium_energy_ts4)):
    if mask_tritium_energy_ts4[aa] == True:
        tS1_ts4.append(S1_ts4[aa])
        tS2_ts4.append(S2_ts4[aa])
        traw_x_ts4.append(raw_x_ts4[aa])
        traw_y_ts4.append(raw_y_ts4[aa])
        traw_r_ts4.append(raw_r_ts4[aa])
        tdrift_time_ts4.append(drift_time_ts4[aa])
        x_del_ts4.append(raw_x_del_ts4[aa])
        y_del_ts4.append(raw_y_del_ts4[aa])
S1_t_ts4 = np.array(tS1_ts4)
S2_t_ts4 = np.array(tS2_ts4)
raw_x_t_ts4 = np.array(traw_x_ts4)
raw_y_t_ts4 = np.array(traw_y_ts4)
raw_r_t_ts4 = np.array(traw_r_ts4)
drift_time_t_ts4 = np.array(tdrift_time_ts4)
negative_drift_time_t_ts4 = [0-a for a in drift_time_t_ts4]
x_del_ts4 = np.array(x_del_ts4)
y_del_ts4 = np.array(y_del_ts4)

In [30]:
# G1 is .104 according to Richard's presentation for Sept 2014
# G2 is 19.15
g1_s4 = 0.104
g2_s4 = 19.15
W = 13.7/1000

S1_energy_t_ts4 = W*S1_t_ts4/g1_s4
S2_energy_t_ts4 = W*S2_t_ts4/g2_s4
Combined_energy_t_ts4 = S1_energy_t_ts4 + S2_energy_t_ts4

In [31]:
Efield_ts4=[]
for ii in xrange(len(x_del_ts4)):
    Efield_ts4.append(GetXYZDependentElectricField((x_del_ts4[ii],y_del_ts4[ii],drift_time_t_ts4[ii]),LoadEFieldMap))
Efield_ts4= np.array(Efield_ts4)

In [32]:
WantToSleep = 1

In [33]:
if(WantToSleep):
    Efield_s2_ts4=[]
    x_real_s2_ts4=[]
    y_real_s2_ts4=[]
    z_real_s2_ts4=[]
    for ii in xrange(len(raw_x_t_ts4)):
        temp =  GetXYZFromXYZS2TDependentElectricField((raw_x_t_ts4[ii],raw_y_t_ts4[ii],drift_time_t_ts4[ii]),LoadEFieldMap)
        x_real_s2_ts4.append(temp[0])
        y_real_s2_ts4.append(temp[1])
        z_real_s2_ts4.append(temp[2])
        Efield_s2_ts4.append(temp[3])
    Efield_s2_ts4= np.array(Efield_s2_ts4)
    x_real_s2_ts4= np.array(x_real_s2_ts4)
    y_real_s2_ts4= np.array(y_real_s2_ts4)
    z_real_s2_ts4= np.array(z_real_s2_ts4)

In [34]:
len(S1_t_ts4)

6117

In [35]:
Efield_s2_ts4

array([  34335.58173655,   42019.1698105 ,   25039.602     , ...,
         34171.21386413,  215502.05061983,  194845.16771452])

In [36]:
mask_S1_ss4 = (ss4.pulse_classification==1)&(ss4.s1s2_pairing==1)
mask_S2_ss4 = (ss4.pulse_classification==2)&(ss4.s1s2_pairing==1)

In [37]:
S1_ss4 = (ss4.xyz_corrected_pulse_area_all_phe*mask_S1_ss4).sum(0)
S2_ss4 = (ss4.xyz_corrected_pulse_area_all_phe*mask_S2_ss4).sum(0)

In [38]:
raw_x_ss4 = (ss4.x_cm*mask_S2_ss4).sum(0) #this is ok since i have verified there is only
raw_y_ss4 = (ss4.y_cm*mask_S2_ss4).sum(0) #one S2 in each event that has made it through cuts
drift_time_ss4 = (ss4.z_drift_samples*mask_S2_ss4).sum(0)/100.
negative_drift_time_ss4 = [0-a for a in drift_time_ss4]
raw_r_ss4 = (raw_x_ss4**2 + raw_y_ss4**2)**(1./2)
#x_del_ss4 = (ss4.x_cm_del*mask_S2_ss4).sum(0)
#y_del_ss4 = (ss4.y_cm_del*mask_S2_ss4).sum(0)
negative_drift_time_ss4 = [0-a for a in drift_time_ss4]
raw_r_ss4 = (raw_x_ss4**2 + raw_y_ss4**2)**(1./2)# mask out only tritium_energies this is a mask on an entire event not a pulse
# possibly add a radial cut
# possibly add a drift time cut
mask_tritium_energy_ss4 = (np.log10(S1_ss4)<2.5)&(np.log10(S2_ss4)<5.0)&(3.1<np.log10(S2_ss4))&(6.1>np.log10(S2_ss4)+np.log10(S1_ss4))
tS1_ss4 = []
tS2_ss4 = []
traw_x_ss4 = []
traw_y_ss4 = []
traw_r_ss4 = []
tdrift_time_ss4 = []
x_del_ss4=[]
y_del_ss4=[]
for aa in range(len(mask_tritium_energy_ss4)):
    if mask_tritium_energy_ss4[aa] == True:
        tS1_ss4.append(S1_ss4[aa])
        tS2_ss4.append(S2_ss4[aa])
        traw_x_ss4.append(raw_x_ss4[aa])
        traw_y_ss4.append(raw_y_ss4[aa])
        traw_r_ss4.append(raw_r_ss4[aa])
        tdrift_time_ss4.append(drift_time_ss4[aa])
       # x_del_ss4.append(raw_x_del_ss4[aa])
       # y_del_ss4.append(raw_y_del_ss4[aa])
S1_t_ss4 = np.array(tS1_ss4)
S2_t_ss4 = np.array(tS2_ss4)
raw_x_t_ss4 = np.array(traw_x_ss4)
raw_y_t_ss4 = np.array(traw_y_ss4)
raw_r_t_ss4 = np.array(traw_r_ss4)
drift_time_t_ss4 = np.array(tdrift_time_ss4)
negative_drift_time_t_ss4 = [0-a for a in drift_time_t_ss4]
x_del_ss4 = np.array(x_del_ss4)
y_del_ss4 = np.array(y_del_ss4)

In [39]:
S1_energy_t_ss4 = W*S1_t_ss4/g1_s4
S2_energy_t_ss4 = W*S2_t_ss4/g2_s4
Combined_energy_t_ss4 = S1_energy_t_ss4 + S2_energy_t_ss4

In [40]:
Efield_t_ss4=[]
for ii in xrange(len(raw_x_t_ss4)):
    try:
        Efield_t_ss4.append(GetXYZDependentElectricField((raw_x_t_ss4[ii],raw_y_t_ss4[ii],drift_time_t_ss4[ii]),LoadEFieldMap))
    except SyntaxError:
        print error
Efield_t_ss4= np.array(Efield_t_ss4)

In [ ]:
raw_x_ss4[3256],raw_y_ss4[3256],drift_time_ss4[3256]

(6.8083115, 1.4315882, 228.52000000000001)

In [ ]:
if(WantToSleep ):
    Efield_s2_ss4=[]
    x_real_s2_ss4=[]
    y_real_s2_ss4=[]
    z_real_s2_ss4=[]
    for ii in xrange(len(raw_x_ss4)):
        temp = GetXYZFromXYZS2TDependentElectricField((raw_x_ss4[ii],raw_y_ss4[ii],drift_time_ss4[ii]),LoadEFieldMap)
        x_real_s2_ss4.append(temp[0])
        y_real_s2_ss4.append(temp[1])
        z_real_s2_ss4.append(temp[2])
        Efield_s2_ss4.append(temp[3])
    Efield_s2_ss4= np.array(Efield_s2_ss4)
    x_real_s2_ss4= np.array(x_real_s2_ss4)
    y_real_s2_ss4= np.array(y_real_s2_ss4)
    z_real_s2_ss4= np.array(z_real_s2_ss4)

In [ ]:
len(ss4)

In [ ]:
WantToSleep = 0

In [ ]:
# S2 vs S1
plt.figure(10)
plt.hist2d(np.log10(S1_ts4),np.log10(S2_ts4),range=[[1,5],[1,6.5]],bins=(100,100),norm=LogNorm())
(mu,sigma,log10xlist)=bandfit(np.log10(S1_ts4),np.log10(S2_ts4),range=[[1,5],[1,6.5]],bins=(50,100),)
plt.plot(log10xlist,mu,lw=4, color="k")
plt.plot(log10xlist,np.array(mu)+np.array(np.abs(sigma)),"k-",lw=2)
plt.plot(log10xlist,np.array(mu)-np.array(np.abs(sigma)),"k-",lw=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("Log10 S1", fontsize=18)
plt.ylabel("Log10 S2", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)

# Traditional band plot
plt.figure(20)
plt.hist2d(S1_ts4, np.log10(S2_ts4)-np.log10(S1_ts4),range=[[0,120],[0,6]],bins=(100,100),norm=LogNorm())
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("S1", fontsize=18)
plt.ylabel("Log10 (S2/S1)", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)

In [ ]:
ts4

In [ ]:
len(S1_t_ts4)

In [ ]:
# S2 vs S1
plt.figure(30)
plt.hist2d(np.log10(S1_t_ts4),np.log10(S2_t_ts4),range=[[1,2.2],[2.8,4.3]],bins=(100,100),norm=LogNorm())
bandfit1=bandfit(np.log10(S1_t_ts4),np.log10(S2_t_ts4),range=[[1,2.2],[2.8,4.3]],bins=(100,100),)
plt.plot(bandfit1[2],bandfit1[0],lw=4, color="k")
plt.plot(bandfit1[2],np.array(bandfit1[0])+np.array(np.abs(bandfit1[1])),"k-",lw=2)
plt.plot(bandfit1[2],np.array(bandfit1[0])-np.array(np.abs(bandfit1[1])),"k-",lw=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("Log10 S1", fontsize=18)
plt.ylabel("Log10 S2", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)

# Traditional band plot
plt.figure(40)
plt.hist2d(S1_t_ts4, np.log10(S2_t_ts4)-np.log10(S1_t_ts4),range=[[0,120],[0,5]],bins=(100,100),norm=LogNorm())
bandfit2=bandfit(S1_t_ts4, np.log10(S2_t_ts4)-np.log10(S1_t_ts4),range=[[0,120],[0,5]],bins=(100,100),)
plt.plot(bandfit2[2],bandfit2[0],lw=4, color="k")
plt.plot(bandfit2[2],np.array(bandfit2[0])+np.array(np.abs(bandfit2[1])),"k-",lw=2)
plt.plot(bandfit2[2],np.array(bandfit2[0])-np.array(np.abs(bandfit2[1])),"k-",lw=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("S1", fontsize=18)
plt.ylabel("Log10 (S2/S1)", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)

In [ ]:
# S2 vs S1
plt.figure(11)
plt.hist2d(np.log10(S1_ss4),np.log10(S2_ss4),range=[[1,2.2],[2.8,4.3]],bins=(100,100),norm=LogNorm())
bandfit01=bandfit(np.log10(S1_ss4),np.log10(S2_ss4),range=[[1,2.2],[2.8,4.3]],bins=(100,100))
plt.plot(bandfit01[2],bandfit01[0],lw=4, color="k")
plt.plot(bandfit01[2],np.array(bandfit01[0])+np.array(np.abs(bandfit01[1])),"k-",lw=2)
plt.plot(bandfit01[2],np.array(bandfit01[0])-np.array(np.abs(bandfit01[1])),"k-",lw=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("Log10 S1", fontsize=18)
plt.ylabel("Log10 S2", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

# Traditional band plot
plt.figure(12)
plt.hist2d(S1_ss4, np.log10(S2_ss4)-np.log10(S1_ss4),range=[[0,120],[0,6]],bins=(100,100),norm=LogNorm())
bandfit02=bandfit(S1_ss4, np.log10(S2_ss4)-np.log10(S1_ss4),range=[[0,120],[0,6]],bins=(100,100))
plt.plot(bandfit02[2],bandfit02[0],lw=4, color="k")
plt.plot(bandfit02[2],np.array(bandfit02[0])+np.array(np.abs(bandfit02[1])),"k-",lw=2)
plt.plot(bandfit02[2],np.array(bandfit02[0])-np.array(np.abs(bandfit02[1])),"k-",lw=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)
plt.xlabel("S1", fontsize=18)
plt.ylabel("Log10 (S2/S1)", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)


In [ ]:
#comparision plot 
plt.figure(12)
plt.errorbar(bandfit1[2],bandfit1[0],bandfit1[1],lw=4, color="k",label="exp")
plt.errorbar(bandfit01[2],bandfit01[0],bandfit01[1],lw=4, color="b",label="sim")
plt.legend(loc=4,fontsize=35)
plt.xlim(1.0,2.3)
plt.ylim(3.0,4.)
plt.xlabel("Log10 S1", fontsize=18)
plt.ylabel("Log10 S2", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)
#comparision plot 
plt.figure(22)
plt.errorbar(bandfit2[2],bandfit2[0],bandfit2[1],lw=4, color="k",label="exp")
plt.errorbar(bandfit02[2],bandfit02[0],bandfit02[1],lw=4, color="b",label="sim")
plt.legend(loc=0,fontsize=35)
plt.xlim(0,100.)
plt.ylim(1.5,4.)
plt.xlabel("S1", fontsize=18)
plt.ylabel("Log10 (S2/S1)", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
# Uncorrected mercury reconstruction
plt.figure(50,figsize=(13,10))
plt.hist2d(raw_x_t_ts4,raw_y_t_ts4,range=[[-25,25],[-25,25]],bins=(100,100))
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
len(raw_y_ss4)

In [ ]:
# Uncorrected mercury reconstruction
plt.figure(51,figsize=(13,10))
plt.hist2d(raw_x_ss4,raw_y_ss4,range=[[-25,25],[-25,25]],bins=(100,100))
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
# corrected reconstruction
plt.figure(60,figsize=(13,10))
plt.hist2d(x_del_ts4,y_del_ts4,range=[[-25,25],[-25,25]],bins=(100,100))
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
# dritft time vs S2R^2
plt.figure(70,figsize=(13,10))
plt.hist2d(raw_r_t_ts4**2,negative_drift_time_t_ts4,range=[[0,25**2],[-400,0]],bins=(70,70))
cbar = plt.colorbar()
plt.xlabel("r^2 (cm^2)",fontsize=18)
plt.ylabel("-drift_time (samples)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
# dritft time vs S2R^2
plt.figure(105,figsize=(13,10))
plt.hist2d(raw_r_ss4**2,negative_drift_time_ss4,range=[[0,25**2],[-400,0]],bins=(100,100))
cbar = plt.colorbar()
plt.xlabel("r^2 (cm^2)",fontsize=18)
plt.ylabel("-drift_time (samples)",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
# dritft time vs S2R^2
plt.figure(80,figsize=(13,10))
plt.hist2d((x_del_ts4**2+y_del_ts4**2),negative_drift_time_t_ts4,range=[[0,25**2],[-400,0]],bins=(70,70))
cbar = plt.colorbar()
plt.xlabel("r^2 (cm^2)",fontsize=18)
plt.ylabel("-drift_time (samples)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
# drift time histogram
plt.figure(90)
plt.hist(drift_time_t_ts4, range=[10.5,380.5], bins=100)
plt.xlabel("drift time (samples)")
plt.title("Tritium Sept 2014", fontsize=20)

# Corrected S1
plt.figure(100)
plt.hist(S1_t_ts4, range=[0,110], bins=110)
plt.xlim(0,110)
plt.xlabel("S1 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

# Corrected S2
plt.figure(101)
plt.hist(S2_t_ts4, range=[0,10500], bins=105)
plt.xlim(0,10500)
plt.xlabel("S2 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
np.linalg.norm(drift_time_ss4[np.where(np.abs(drift_time_ss4-195.5)<185.5)])

In [ ]:
from scipy.stats import ks_2samp
ks_2samp(drift_time_ss4[np.where(np.abs(drift_time_ss4-195.5)<185.5)]/np.linalg.norm(drift_time_ss4[np.where(np.abs(drift_time_ss4-195.5)<185.5)]),\
         drift_time_t_ts4[np.where(np.abs(drift_time_t_ts4-195.5)<185.5)]/np.linalg.norm(drift_time_t_ts4[np.where(np.abs(drift_time_t_ts4-195.5)<185.5)]))

In [ ]:
# drift time histogram
plt.figure(91)
plt.hist(drift_time_t_ss4, range=[10.5,380.5], bins=100)
plt.xlabel("drift time (samples)", fontsize=18)

# Corrected S1
plt.figure(101)
plt.hist(S1_t_ss4, range=[-0.5,110.5], bins=111)
plt.xlim(0,110)
plt.xlabel("S1 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

# Corrected S2
plt.figure(111)
plt.hist(S2_t_ss4, range=[0,10500], bins=105)
plt.xlim(0,10500)
plt.xlabel("S2 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
# combined energy
plt.figure(120)
plt.hist(Combined_energy_t_ts4, range=[0,20], bins=60)
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)
plt.figure(121)
plt.hist(Combined_energy_t_ss4, range=[0,20], bins=60)
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.title("Tritium Sim Sept 2015", fontsize=20)

In [ ]:
# combined energy
plt.figure(122)
#ksTestValue =sp.stats.ks_2samp(Combined_energy_t_ss4,Combined_energy_t_ts4)
print ksTestValue
#plt.text(.5,.5,"KS Test Value = %g"% ksTestValue[1])
plt.hist(Combined_energy_t_ss4, normed =1,range=[0,20], bins=60, label = "sim")
plt.hist(Combined_energy_t_ts4, normed =1,range=[0,20], bins=60, label = "exp")
plt.legend()
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.title("Tritium Sept 2015", fontsize=20)
plt.title("Tritium Sim Sept 2015", fontsize=20)

In [ ]:
plt.figure(130)
plt.hist2d(Combined_energy_t_ts4, S1_t_ts4/Combined_energy_t_ts4,range=[[0,20],[0,6]],bins=(100,100),norm=LogNorm())
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.ylabel("S1 Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

In [ ]:
plt.figure(131)
plt.hist2d(Combined_energy_ss4, S1_ss4/Combined_energy_ss4,range=[[0,20],[0,6]],bins=(100,100),norm=LogNorm())
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.ylabel("S1 pulse ", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

In [ ]:
# light yield
plt.figure(132)
bandfit10=bandfit(Combined_energy_t_ss4, S1_t_ss4/Combined_energy_t_ss4,range=[[0,20],[0,7]],bins=(20,20),)
bandfit010=bandfit(Combined_energy_t_ts4, S1_t_ts4/Combined_energy_t_ts4,range=[[0,20],[0,7]],bins=(20,20),)
plt.errorbar(bandfit10[2],bandfit10[0],bandfit10[1],lw=2, color="k",label="s1 sim")
plt.errorbar(bandfit010[2],bandfit010[0],bandfit010[1],lw=2, color="b",label="s1 exp")
bandfits = np.load("LYenergygaussian.npy")
plt.errorbar(bandfits[2]*100.,bandfits[0],bandfits[1],lw=4, color="g",label="LibNEST gaussian fit bins")

plt.legend(loc=0,fontsize=35)
plt.xlim(0,20.)
#plt.ylim(,)
plt.legend()
plt.xlabel("Deposited Energy (keV ee)", fontsize=18)
plt.ylabel("S1 / Deposited Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2015", fontsize=20)


In [ ]:
plt.figure(140)
plt.hist2d(Efield_ts4[~np.isnan(Efield_ts4)], S1_t_ts4[~np.isnan(Efield_ts4)]/Combined_energy_t_ts4[~np.isnan(Efield_ts4)],range=[[0,70000],[0,8]],bins=(100,100),norm=LogNorm())
bandfits140=bandfit(Efield_ts4[~np.isnan(Efield_ts4)], S1_t_ts4[~np.isnan(Efield_ts4)]/Combined_energy_t_ts4[~np.isnan(Efield_ts4)],range=[[0,70000],[0,8]],bins=(100,100))
plt.errorbar(bandfits140[2],bandfits140[0],bandfits140[1],lw=2, color="k",label="s1 sim")
plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2015", fontsize=20)

In [ ]:
plt.figure(141)
plt.hist2d(Efield_t_ss4[~np.isnan(Efield_t_ss4)], S1_t_ss4[~np.isnan(Efield_t_ss4)]/Combined_energy_t_ss4[~np.isnan(Efield_t_ss4)],range=[[0,70000],[0,8]],bins=(100,100),norm=LogNorm())
bandfits141=bandfit(Efield_t_ss4[~np.isnan(Efield_t_ss4)], S1_t_ss4[~np.isnan(Efield_t_ss4)]/Combined_energy_t_ss4[~np.isnan(Efield_t_ss4)],range=[[0,70000],[0,8]],bins=(100,100))
plt.errorbar(bandfits141[2],bandfits141[0],bandfits141[1],lw=2, color="k",label="s1 sim")

plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

In [ ]:
plt.figure(142)
plt.errorbar(bandfits140[2],bandfits140[0],bandfits140[1],lw=2, color="k",label="s1 exp")
plt.errorbar(bandfits141[2],bandfits141[0],bandfits141[1],lw=2, color="b",label="s1 sim")
plt.legend()
plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

In [ ]:
plt.figure(150)
plt.hist2d(Efield_s2_ts4[~np.isnan(Efield_s2_ts4)], S1_t_ts4[~np.isnan(Efield_s2_ts4)]/Combined_energy_t_ts4[~np.isnan(Efield_s2_ts4)],range=[[0,70000],[0,8]],bins=(100,100),norm=LogNorm())
bandfits150=bandfit(Efield_s2_ts4[~np.isnan(Efield_s2_ts4)], S1_t_ts4[~np.isnan(Efield_s2_ts4)]/Combined_energy_t_ts4[~np.isnan(Efield_s2_ts4)],range=[[0,70000],[0,8]],bins=(100,100))
plt.errorbar(bandfits150[2],bandfits150[0],bandfits150[1],lw=2, color="k",label="s1 sim")

plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

plt.figure(151)
plt.hist2d(Efield_s2_ss4[~np.isnan(Efield_s2_ss4)], S1_ss4[~np.isnan(Efield_s2_ss4)]/Combined_energy_ss4[~np.isnan(Efield_s2_ss4)],range=[[0,70000],[0,8]],bins=(100,100),norm=LogNorm())
bandfits151=bandfit(Efield_s2_ss4[~np.isnan(Efield_s2_ss4)], S1_ss4[~np.isnan(Efield_s2_ss4)]/Combined_energy_ss4[~np.isnan(Efield_s2_ss4)],range=[[0,70000],[0,8]],bins=(100,100))
plt.errorbar(bandfits151[2],bandfits151[0],bandfits151[1],lw=2, color="k",label="s1 sim")

plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)
plt.figure(152)
plt.errorbar(bandfits150[2],bandfits150[0],bandfits150[1],lw=2, color="k",label="s1 exp")
plt.errorbar(bandfits151[2],bandfits151[0],bandfits151[1],lw=2, color="b",label="s1 sim")
plt.legend()
plt.xlabel("E field (V/m)", fontsize=18)
plt.ylabel("S1 /Deposit Energy", fontsize=18)
plt.title("Tritium Comparision Sept 2014", fontsize=20)

In [ ]:
np.seterr(invalid='ignore')
# corrected reconstruction
plt.figure(160,figsize=(13,10))
plt.hist2d(x_real_s2_ts4,y_real_s2_ts4,range=[[-25,25],[-25,25]],bins=(100,100))
plt.clim(0,15)
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

# z vs S2R^2
plt.figure(170,figsize=(13,10))
plt.hist2d(x_real_s2_ts4**2+y_real_s2_ts4**2,z_real_s2_ts4,range=[[0,25**2],[0,60]],bins=(70,70))
plt.clim(0,30)
cbar = plt.colorbar()
plt.xlabel("r^2 (cm^2)",fontsize=18)
plt.ylabel("-drift_time (samples)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
# corrected reconstruction
plt.figure(161,figsize=(13,10))
plt.hist2d(x_real_s2_ss4,y_real_s2_ss4,range=[[-25,25],[-25,25]],bins=(100,100))
plt.clim(0,5)
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)
# z vs S2R^2
plt.figure(170,figsize=(13,10))
plt.hist2d(x_real_s2_ss4**2+y_real_s2_ss4**2,z_real_s2_ss4,range=[[0,25**2],[0,60]],bins=(70,70))
plt.clim(0,10)
cbar = plt.colorbar()
plt.xlabel("r^2 (cm^2)",fontsize=18)
plt.ylabel("-drift_time (samples)",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
plt.figure(160,figsize=(13,10))
plt.hist2d(x_real_s2_ts4,y_real_s2_ts4,range=[[-25,25],[-25,25]],bins=(100,100))
plt.clim(0,15)
cbar = plt.colorbar()
plt.xlabel("S2_X (cm)", fontsize=18)
plt.ylabel("S2_Y (cm)", fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)

In [ ]:
# Corrected S1
plt.figure(991)
plt.hist(S2_t_ts4[np.where(np.abs(S1_t_ts4-3)<0.5)], range=[1000,4000], bins=111)
#plt.xlim(0,110)
plt.xlabel("S2 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)
# Corrected S1
plt.figure(992)
plt.hist(S2_t_ss4[np.where(np.abs(S1_t_ss4-3)<0.5)], range=[1000,4000], bins=111)
#plt.xlim(0,110)
plt.xlabel("S2 Corrected (ph?)",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)
# Corrected S1
plt.figure(993)
plt.hist(z_real_s2_ts4[np.where(np.abs(S1_t_ts4-3)<0.5)], range=[0,60], bins=100)
#plt.xlim(0,110)
plt.xlabel("z",fontsize=18)
plt.title("Tritium Sept 2014", fontsize=20)
# Corrected S1
plt.figure(994)
plt.hist(z_real_s2_ss4[np.where(np.abs(S1_t_ss4-3)<0.5)], range=[0,60], bins=100)
#plt.xlim(0,110)
plt.xlabel("z",fontsize=18)
plt.title("Tritium Sim Sept 2014", fontsize=20)

In [ ]:
filedir = "/project/projectdirs/lux/users/weiji/band/"
np.save(filedir+"201509explogband",bandfit1)
np.save(filedir+"201509simlogband",bandfit01)
np.save(filedir+"201509expband",bandfit2)
np.save(filedir+"201509simband",bandfit02)
np.save(filedir+"201509explightyield",bandfits150)
np.save(filedir+"201509simlightyield",bandfits151)

In [ ]:
len(S1_ss4)

In [ ]:
S1_ss4[~np.isnan(Efield_s2_ss4)]

In [ ]:
~np.isnan(Efield_s2_ss4)

In [ ]:
Efield_s2_ss4

In [ ]:
drift_time_ss4

In [ ]:
drift_time_t_ts4